### Inverse UQ of m parameter

The goal of this script is to determine the data informed posterior of the parameter m using the Fisher approcimation.

In [1]:
using Pkg
Pkg.activate("C:/Users/lisah/Documents/Repos/ma-code")

  Activating project at `C:\Users\lisah\Documents\Repos\ma-code`


In [30]:
include("c:/Users/lisah/Documents/Repos/ma-code/src/src.jl")
using .Src, DataFrames, Optim, ForwardDiff, LinearAlgebra, CSV

In [26]:
"""
    function compute_ll_m(x, hprm::Hyperprm, true_val::DataFrame; t_fixed::Bool=false, t_end::Float64=50.0, t_step::Float64=1.0)

compute the log-likelihood with m being parameter of interest.         
"""
function compute_ll_m(x, hprm::Src.Hyperprm, true_val::DataFrame; t_fixed::Bool=false, t_end::Float64=50.0, t_step::Float64=1.0)
    m = x[1]
    hprm = Src.Hyperprm(hprm.w0, hprm.n0, hprm.a, m, hprm.M, hprm.noise)
    pred_val = Src.sol_klausmeier(hprm; t_fixed=t_fixed, t_end=t_end, t_step=t_step)
    if hprm.noise == 0.0
        ll = -0.5 * sum((true_val[:,"n"] - pred_val[:,"n"]) .^2) - 0.5 * sum((true_val[:,"w"] - pred_val[:,"w"]) .^2) # add up ll for both trajectories
    else
        ll = -0.5 * 1/hprm.noise * sum((true_val[:,"n"] - pred_val[:,"n"]) .^2) - 0.5 * 1/hprm.noise * sum((true_val[:,"w"] - pred_val[:,"w"]) .^2) # add up ll for both trajectories
    end
    return ll
end

"""
    function compute_mle(hprm::Hyperprm, true_val::DataFrame; t_fixed::Bool=false, t_end::Float64=50.0, t_step::Float64=1.0)
"""
function compute_mle_m(hprm::Src.Hyperprm, true_val::DataFrame; t_fixed::Bool=false, t_end::Float64=50.0, t_step::Float64=1.0)
    result = optimize(x -> -compute_ll_m(x, hprm, true_val; t_fixed=t_fixed, t_end=t_end, t_step=t_step), 0.45-0.45*2, 0.45+0.45*2, Brent()) # initialize optimization at true prm values s.th. global min is found
    return Optim.minimizer(result), Optim.converged(result)
end

"""
    function compute_fi(eval_pt::Vector{Float64}, hprm::Hyperprm, true_val::DataFrame; t_fixed::Bool=false, t_end::Float64=50.0, t_step::Float64=1.0)
"""
function compute_fi_m(eval_pt::Vector{Float64}, hprm::Src.Hyperprm, true_val::DataFrame; t_fixed::Bool=false, t_end::Float64=50.0, t_step::Float64=1.0)
    H = ForwardDiff.hessian(x -> compute_ll_m(x, hprm, true_val; t_fixed=t_fixed, t_end=t_end, t_step=t_step), eval_pt)
    return tr(-H)
end

function store_fish_data(w0::Float64,M::Int64,noise::Float64,df::DataFrame, path::String)
    CSV.write("$(path)fish_$(w0)_$(M)_$(noise).csv", df)
end

function store_mle_data(w0::Float64,M::Int64,noise::Float64,df::DataFrame, path::String)
    CSV.write("$(path)mle_$(w0)_$(M)_$(noise).csv", df)
end

function gen_all_fish_data_m(M_vals, noise_vals, m, w0, path; t_fixed::Bool=false, t_end::Float64=50.0, t_step::Float64=1.0)
    for M in M_vals
        for noise in noise_vals

            grid = Src.create_grid()
            fish = zeros(41, 21)
            mle_m = zeros(41,21)

            # keep track of whether the optimization algo terminates successfully when finding the MLE
            success_counter = 0
            eval_pt_counter = 0

            # evaluate fisher info on grid
            for i in range(1, 41)
                for j in range(1, 21)
                    eval_pt_counter = eval_pt_counter + 1 # total number of optimizations

                    pt = grid[i,j] # true observation parameter point
                    hprm = Src.Hyperprm(w0, pt[2], pt[1], m, M, noise) # w0,n0,a,m,M

                    sol_true = Src.sol_klausmeier(hprm; t_fixed=t_fixed, t_end=t_end, t_step=t_step)
                    sol_true = Src.randomize_data(sol_true, hprm.noise) # include noise

                    if noise == 0.0 # in this case stick to true prm combination
                        mle, success = hprm.m, true
                    else
                        mle, success = compute_mle_m(hprm, sol_true; t_fixed=t_fixed, t_end=t_end, t_step=t_step)
                    end

                    # store mles
                    mle_m[i,j] = mle

                    # evaluate Fi at MLE
                    fish[i,j] = compute_fi_m([mle], hprm, sol_true; t_fixed=t_fixed, t_end=t_end, t_step=t_step)

                    success_counter = success_counter + success # number of successfull optimizations
                end
            end

            success_fraction = success_counter / eval_pt_counter
            println("MLE terminated with success in $success_fraction cases.")
            
            # create data frame
            a_eval_pts = string.(0.0:0.1:2.0)
            df_fish = DataFrame(fish, a_eval_pts)
            df_mle = DataFrame(mle_m, a_eval_pts)

            store_fish_data(w0, M, noise, df_fish, path)
            store_mle_data(w0, M, noise, df_mle, path)
        end
    end
end

gen_all_fish_data_m (generic function with 1 method)

In [27]:
# general setup for analysis
t_fixed=true
t_end=500.0

500.0

In [ ]:
# experiment for some prm
w0=0.95
n0=1.9
a=1.0
m=0.45 # true m underlying data observations
M=1000
noise=1.0
hprm = Src.Hyperprm(w0,n0,a,m,M,noise)
data_true = Src.sol_klausmeier(hprm)
data_obs = Src.randomize_data(data_true, noise)

post_mean = compute_mle_m(hprm, data_obs; t_fixed, t_end)[1] # inferred m by Fisher approximation (given observed noisy data)
fi = compute_fi_m([post_mean], hprm, data_obs, t_fixed=t_fixed, t_end=t_end) # confidence level of inferred m

(post_mean, fi)

(0.4461808923740412, 67461.444732726)

In [31]:
path = "C:/Users/lisah/Documents/Repos/ma-code/data/t_fixed/t500/fisher_m/"
noise_vals = [0.1, 1.0]
M_vals = [10,100,500,1000]
w0 = 0.95
m = 0.45

gen_all_fish_data_m(M_vals, noise_vals, m, w0, path, t_fixed=t_fixed, t_end=t_end)

MLE terminated with success in 1.0 cases.
MLE terminated with success in 1.0 cases.
MLE terminated with success in 1.0 cases.
MLE terminated with success in 1.0 cases.
MLE terminated with success in 1.0 cases.
MLE terminated with success in 1.0 cases.
MLE terminated with success in 1.0 cases.
MLE terminated with success in 1.0 cases.
